In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime



In [2]:
housing_prices = pd.read_csv('Resources/housing_prices.csv')
housing_prices

,Date,Area,Composite_HPI,Two_Storey_HPI,Apartment_HPI,Composite_Benchmark,Two_Storey_Benchmark,Apartment_Benchmark
0,Jan 2005,Barrie_and_District,100.0,100.0,100.0,209000.0,220600.0,166400.0
1,Feb 2005,Barrie_and_District,101.5,101.2,100.3,212100.0,223200.0,166900.0
2,Mar 2005,Barrie_and_District,102.8,102.9,100.6,214800.0,227000.0,167400.0
3,Apr 2005,Barrie_and_District,103.5,103.4,100.9,216300.0,228100.0,167900.0
4,May 2005,Barrie_and_District,104.5,104.0,101.2,218400.0,229400.0,168400.0
...,...,...,...,...,...,...,...,...
1119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
income = pd.read_csv('Resources/Income - Toronto.csv')
income.head()

,REF_DATE,GEO,DGUID,Sex,Age group,Persons with income,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2000,"Toronto, Ontario",2016S0503535,Both sexes,All age groups,Total persons with income,Number,223,units,0,v20797165,19.1.1.1,3371060,NaN,NaN,NaN,0
1,2000,"Toronto, Ontario",2016S0503535,Both sexes,All age groups,"Persons with income under $5,000",Number,223,units,0,v20797166,19.1.1.2,415550,NaN,NaN,NaN,0
2,2000,"Toronto, Ontario",2016S0503535,Both sexes,All age groups,"Persons with income of $5,000 and over",Number,223,units,0,v20797167,19.1.1.3,2955520,NaN,NaN,NaN,0
3,2000,"Toronto, Ontario",2016S0503535,Both sexes,All age groups,"Persons with income of $10,000 and over",Number,223,units,0,v20797168,19.1.1.4,2584590,NaN,NaN,NaN,0
4,2000,"Toronto, Ontario",2016S0503535,Both sexes,All age groups,"Persons with income of $15,000 and over",Number,223,units,0,v20797169,19.1.1.5,2204370,NaN,NaN,NaN,0


In [4]:
housing_prices = housing_prices.dropna()
housing_prices.shape

(1110, 8)

Original database was relatively clean. Only 14 rows contained na values 

In [5]:
housing_prices[['Month','Year']] = housing_prices.Date.str.split(" ", expand=True)
housing_prices.head()

C:\Users\dlefo\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Date,Area,Composite_HPI,Two_Storey_HPI,Apartment_HPI,Composite_Benchmark,Two_Storey_Benchmark,Apartment_Benchmark,Month,Year
0,Jan 2005,Barrie_and_District,100.0,100.0,100.0,209000.0,220600.0,166400.0,Jan,2005
1,Feb 2005,Barrie_and_District,101.5,101.2,100.3,212100.0,223200.0,166900.0,Feb,2005
2,Mar 2005,Barrie_and_District,102.8,102.9,100.6,214800.0,227000.0,167400.0,Mar,2005
3,Apr 2005,Barrie_and_District,103.5,103.4,100.9,216300.0,228100.0,167900.0,Apr,2005
4,May 2005,Barrie_and_District,104.5,104.0,101.2,218400.0,229400.0,168400.0,May,2005


In [18]:
clean_housing_prices = housing_prices[['Area','Year','Month','Composite_Benchmark','Two_Storey_Benchmark','Apartment_Benchmark']]
clean_housing_prices

,Area,Year,Month,Composite_Benchmark,Two_Storey_Benchmark,Apartment_Benchmark
0,Barrie_and_District,2005,Jan,209000.0,220600.0,166400.0
1,Barrie_and_District,2005,Feb,212100.0,223200.0,166900.0
2,Barrie_and_District,2005,Mar,214800.0,227000.0,167400.0
3,Barrie_and_District,2005,Apr,216300.0,228100.0,167900.0
4,Barrie_and_District,2005,May,218400.0,229400.0,168400.0
...,...,...,...,...,...,...
1105,Oakville_Milton,2020,Jan,1048400.0,1100600.0,553600.0
1106,Oakville_Milton,2020,Feb,1078000.0,1137900.0,555400.0
1107,Oakville_Milton,2020,Mar,1113200.0,1177500.0,565900.0
1108,Oakville_Milton,2020,Apr,1096900.0,1149300.0,574900.0


In [ ]:
# bin = ['Winter','Spring','Summer','Fall']

# bin_seasons = 